# 흐름 요약
*	1.	🎥 비디오 ➝ 프레임 추출 (.jpg)
*	2.	📍 MediaPipe로 33개 관절 추출
*	3.	🔁 YOLOv8-pose용 JSON/TXT로 변환
*	4.	📁 디렉터리 구조 & YAML 구성
*	5.	🚀 yolo pose train 으로 학습

## 1~3 비디오 ➝ MediaPipe로 33개 관절 추출 ➝ YOLOv8-pose용 JSON/TXT로 변환
* 일정 프레임 간격으로 이미지 생성
* 보정값사용하면 실제움직임보다 추정값이 느리게 따라옴

In [ ]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import numpy as np

# ===== 설정 =====
video_dir = "climbvideo"
model_path = "pose_landmarker_heavy.task"
output_image_dir = "output/images"
output_label_dir = "output/labels"
FRAME_INTERVAL = 10

# ===== 디렉토리 생성 =====
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# ===== MediaPipe Pose 모델 초기화 =====
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=model_path),
    output_segmentation_masks=False,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1
)
landmarker = vision.PoseLandmarker.create_from_options(options)

# ===== 연결관계 정의 (11~32번 관절) =====
POSE_CONNECTIONS = [
    (11,12), (11,13), (13,15), (12,14), (14,16),
    (11,23), (12,24), (23,24), (23,25), (24,26),
    (25,27), (26,28), (27,29), (28,30), (29,31), (30,32)
]

# ===== 전체 비디오 파일 목록 =====
video_files = sorted([f for f in os.listdir(video_dir) if f.endswith(".mp4")])
frame_index = 0  # 전역 프레임 인덱스
timestamp = 0    # 전역 timestamp (μs)

# ===== 각 비디오 순회 =====
for video_name in video_files:
    print(f"🎥 처리 시작: {video_name}")
    video_path = os.path.join(video_dir, video_name)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        timestamp += int(1e6 / fps)  # 프레임마다 timestamp 증가

        if frame_index % FRAME_INTERVAL != 0:
            frame_index += 1
            continue

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)
        result = landmarker.detect_for_video(mp_image, timestamp)

        if result.pose_landmarks:
            lm = result.pose_landmarks[0]
            raw_coords = [(pt.x, pt.y, pt.z) for pt in lm]
            filtered = [raw_coords[i] for i in range(11, 33)]  # 얼굴 제외

            # 라벨 생성
            keypoints = []
            x_vals, y_vals = [], []
            for x, y, z in filtered:
                v = 1  # visibility 무시하고 항상 1
                x_vals.append(x)
                y_vals.append(y)
                keypoints.extend([x, y, v])

            # 바운딩 박스 계산
            x_min = max(min(x_vals), 0)
            y_min = max(min(y_vals), 0)
            x_max = min(max(x_vals), 1)
            y_max = min(max(y_vals), 1)
            cx = (x_min + x_max) / 2
            cy = (y_min + y_max) / 2
            bw = x_max - x_min
            bh = y_max - y_min

            # YOLOv8 포맷 저장
            label_line = [0, cx, cy, bw, bh] + keypoints
            label_str = " ".join([f"{x:.6f}" for x in label_line])
            label_path = os.path.join(output_label_dir, f"{frame_index:08}.txt")
            with open(label_path, 'w') as f:
                f.write(label_str + "\n")

            # 시각화 후 이미지 저장
            for (x, y, _) in filtered:
                cx_i = int(x * frame.shape[1])
                cy_i = int(y * frame.shape[0])
                cv2.circle(frame, (cx_i, cy_i), 3, (0, 255, 0), -1)

            for start, end in POSE_CONNECTIONS:
                s = start - 11
                e = end - 11
                if s < 0 or e < 0 or s >= len(filtered) or e >= len(filtered):
                    continue
                x1 = int(filtered[s][0] * frame.shape[1])
                y1 = int(filtered[s][1] * frame.shape[0])
                x2 = int(filtered[e][0] * frame.shape[1])
                y2 = int(filtered[e][1] * frame.shape[0])
                cv2.line(frame, (x1, y1), (x2, y2), (255, 255, 255), 1)

            image_path = os.path.join(output_image_dir, f"{frame_index:08}.jpg")
            cv2.imwrite(image_path, frame)

        frame_index += 1

    cap.release()
    print(f"✅ 완료: {video_name}")

print("🎯 전체 비디오 처리 완료!")

I0000 00:00:1746151354.630362 1016283 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1746151354.696849 1020205 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746151354.740185 1020203 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746151354.854080 1020204 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


🎥 처리 시작: IMG_0208.mp4
✅ 완료: IMG_0208.mp4
🎥 처리 시작: IMG_0211.mp4
✅ 완료: IMG_0211.mp4
🎥 처리 시작: IMG_0218.mp4
✅ 완료: IMG_0218.mp4
🎥 처리 시작: IMG_0231.mp4
✅ 완료: IMG_0231.mp4
🎥 처리 시작: IMG_0245.mp4
✅ 완료: IMG_0245.mp4
🎥 처리 시작: IMG_0249.mp4
✅ 완료: IMG_0249.mp4
🎥 처리 시작: IMG_0258.mp4


### 1~3에서 만들어진 이미지 직접 필터링
* 걸러낸 이미지에 맞게 라벨링파일 삭제
* 준비된 이미지와 라벨링파일을 하나의 디렉토리에 저장
    * 중복되면 _00000001 식으로 붙여서 중복방지 / 라벨링파일도 마찬가지

In [1]:
import os

# 경로 설정
image_dir = "output/images"
label_dir = "output/labels"

# 남겨둘 프레임 이름 목록 추출 (예: frame_0042)
image_basenames = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith(".jpg")}

# 라벨 디렉토리 내 .txt 중에서 이미지가 없는 건 삭제
for txt_file in os.listdir(label_dir):
    if not txt_file.endswith(".txt"):
        continue
    base_name = os.path.splitext(txt_file)[0]
    if base_name not in image_basenames:
        os.remove(os.path.join(label_dir, txt_file))
        print(f"🗑 삭제됨: {txt_file}")

print("✅ 완료: 이미지에 해당하는 라벨만 남김")

🗑 삭제됨: 00000700.txt
🗑 삭제됨: 00004200.txt
🗑 삭제됨: 00016610.txt
🗑 삭제됨: 00009730.txt
🗑 삭제됨: 00012310.txt
🗑 삭제됨: 00017240.txt
🗑 삭제됨: 00008360.txt
🗑 삭제됨: 00013740.txt
🗑 삭제됨: 00021080.txt
🗑 삭제됨: 00001350.txt
🗑 삭제됨: 00025580.txt
🗑 삭제됨: 00005650.txt
🗑 삭제됨: 00022210.txt
🗑 삭제됨: 00007090.txt
🗑 삭제됨: 00003590.txt
🗑 삭제됨: 00011180.txt
🗑 삭제됨: 00015480.txt
🗑 삭제됨: 00012660.txt
🗑 삭제됨: 00016360.txt
🗑 삭제됨: 00009240.txt
🗑 삭제됨: 00004770.txt
🗑 삭제됨: 00000270.txt
🗑 삭제됨: 00005320.txt
🗑 삭제됨: 00001620.txt
🗑 삭제됨: 00004980.txt
🗑 삭제됨: 00013230.txt
🗑 삭제됨: 00017730.txt
🗑 삭제됨: 00008610.txt
🗑 삭제됨: 00012890.txt
🗑 삭제됨: 00022760.txt
🗑 삭제됨: 00019090.txt
🗑 삭제됨: 00026260.txt
🗑 삭제됨: 00023330.txt
🗑 삭제됨: 00022990.txt
🗑 삭제됨: 00000310.txt
🗑 삭제됨: 00004610.txt
🗑 삭제됨: 00009320.txt
🗑 삭제됨: 00016200.txt
🗑 삭제됨: 00012700.txt
🗑 삭제됨: 00008980.txt
🗑 삭제됨: 00008770.txt
🗑 삭제됨: 00017650.txt
🗑 삭제됨: 00013350.txt
🗑 삭제됨: 00001740.txt
🗑 삭제됨: 00021490.txt
🗑 삭제됨: 00025190.txt
🗑 삭제됨: 00026300.txt
🗑 삭제됨: 00022600.txt
🗑 삭제됨: 00007480.txt
🗑 삭제됨: 00023250.txt


* 파일이동

In [26]:
import os
import shutil

def move_pair_with_suffix(image_path, label_path, dst_image_dir, dst_label_dir):
    base_name, _ = os.path.splitext(os.path.basename(image_path))  # 예: 00000042
    attempt = 0
    new_base = base_name

    while True:
        img_dst = os.path.join(dst_image_dir, new_base + ".jpg")
        txt_dst = os.path.join(dst_label_dir, new_base + ".txt")
        if not os.path.exists(img_dst) and not os.path.exists(txt_dst):
            break
        attempt += 1
        suffix = f"_{attempt:08d}"
        new_base = base_name + suffix

    shutil.move(image_path, img_dst)
    shutil.move(label_path, txt_dst)
    print(f"✅ moved: {img_dst}, {txt_dst}")

# ==================== 다수 파일 이동 ====================
src_img_dir = "output/images"
src_txt_dir = "output/labels"
dst_img_dir = "/Users/laxdin24/Desktop/train/images"
dst_txt_dir = "/Users/laxdin24/Desktop/train/labels"

os.makedirs(dst_img_dir, exist_ok=True)
os.makedirs(dst_txt_dir, exist_ok=True)

for fname in os.listdir(src_img_dir):
    if not fname.endswith(".jpg"):
        continue
    base = os.path.splitext(fname)[0]
    img_path = os.path.join(src_img_dir, base + ".jpg")
    txt_path = os.path.join(src_txt_dir, base + ".txt")

    if os.path.exists(txt_path):  # 쌍이 맞는 경우만
        move_pair_with_suffix(img_path, txt_path, dst_img_dir, dst_txt_dir)

✅ moved: /Users/laxdin24/Desktop/train/images/00000400.jpg, /Users/laxdin24/Desktop/train/labels/00000400.txt
✅ moved: /Users/laxdin24/Desktop/train/images/00000170.jpg, /Users/laxdin24/Desktop/train/labels/00000170.txt
✅ moved: /Users/laxdin24/Desktop/train/images/00000010.jpg, /Users/laxdin24/Desktop/train/labels/00000010.txt
✅ moved: /Users/laxdin24/Desktop/train/images/00000560_00000001.jpg, /Users/laxdin24/Desktop/train/labels/00000560_00000001.txt
✅ moved: /Users/laxdin24/Desktop/train/images/00000210_00000001.jpg, /Users/laxdin24/Desktop/train/labels/00000210_00000001.txt
✅ moved: /Users/laxdin24/Desktop/train/images/00000600.jpg, /Users/laxdin24/Desktop/train/labels/00000600.txt
✅ moved: /Users/laxdin24/Desktop/train/images/00000370.jpg, /Users/laxdin24/Desktop/train/labels/00000370.txt
✅ moved: /Users/laxdin24/Desktop/train/images/00000360.jpg, /Users/laxdin24/Desktop/train/labels/00000360.txt
✅ moved: /Users/laxdin24/Desktop/train/images/00000610.jpg, /Users/laxdin24/Desktop/

## 4. 디렉터리 구조 & YAML 구성
### train/val 자동 분리 스크립트 + data.yaml 생성

In [ ]:
import os
import shutil
import random

# 경로 설정
image_dir = "output/images"
label_dir = "output/labels"
output_base = "yolo_climber_pose_dataset"
train_img_dir = os.path.join(output_base, "images/train")
val_img_dir = os.path.join(output_base, "images/val")
train_lbl_dir = os.path.join(output_base, "labels/train")
val_lbl_dir = os.path.join(output_base, "labels/val")
yaml_path = os.path.join(output_base, "data.yaml")

# data.yaml 생성
with open(yaml_path, "w") as f:
    f.write(f"""\
path: {output_base}
train: images/train
val: images/val

kpt_shape: [1, 22, 3]  # 1명, 21 keypoints, (x, y, v)
names: ['climber']
""")

print(f"✅ data.yaml 생성 완료: {yaml_path}")

# 디렉토리 생성
for d in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# .jpg 파일 목록 불러오기
images = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])
random.shuffle(images)

# 8:2 split
split_idx = int(len(images) * 0.8)
train_imgs = images[:split_idx]
val_imgs = images[split_idx:]

def move_pair(img_list, img_dst, lbl_dst):
    for img in img_list:
        base = os.path.splitext(img)[0]
        img_src = os.path.join(image_dir, img)
        lbl_src = os.path.join(label_dir, base + ".txt")
        shutil.copy(img_src, os.path.join(img_dst, img))
        shutil.copy(lbl_src, os.path.join(lbl_dst, base + ".txt"))

move_pair(train_imgs, train_img_dir, train_lbl_dir)
move_pair(val_imgs, val_img_dir, val_lbl_dir)

print(f"✅ 데이터셋 분리 완료: {len(train_imgs)} train / {len(val_imgs)} val")

✅ 데이터셋 분리 완료: 639 train / 160 val


## 5. yolo pose train 으로 학습
* Ultralytics 학습 명령어

### 📊 YOLOv8 Pose 모델별 최소 권장 이미지 수

| 모델           | 최소 권장 이미지 수        | 권장 사용 상황                  |
|----------------|-----------------------------|----------------------------------|
| `yolov8n-pose` | 500 ~ 1,000장               | 실험용, 속도 우선, 가벼운 모델   |
| `yolov8s-pose` | 1,000 ~ 3,000장             | 소규모 데이터, 빠른 추론        |
| `yolov8m-pose` | ✅ 3,000 ~ 10,000장         | 중간 규모 학습, 좋은 정확도     |
| `yolov8l/x-pose` | 10,000장 이상 (대규모 권장) | 높은 정확도 필요, 자원 여유 시   |
 

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-pose.pt')  # YOLOv8 모델 로드

model.train(
    data=r"C:\Users\USER\Desktop\yolo_climber_pose_dataset\yolo_climber_pose_dataset\data.yaml",
    epochs=100,
    imgsz=640,
    device='0',  # ✅ device='0' : 너의 NVIDIA GPU로 학습 진행
    # resume=True,  # ✅ resume=True : 이전 학습을 이어서 진행
)

## Best 모델로 추론하기

In [32]:
from ultralytics import YOLO

# 1. 학습된 best.pt 모델 로드
model = YOLO("/Users/laxdin24/Downloads/best.pt")  # ✅ 학습된 모델 경로

# 2. 비디오에서 자세 추정
model.predict(
    source="/Users/laxdin24/Downloads/KakaoTalk_Video_2025-05-01-15-02-18.mp4",  # ✅ 추론할 비디오 파일 경로
    imgsz=640,                # 입력 크기
    conf=0.2,                 # confidence threshold
    save=True,                # 결과 비디오 저장
    save_txt=False,           # 라벨 파일 저장 여부 (원하면 True)
    device='cpu'                # GPU 사용 (없으면 'cpu')
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1318) /Users/laxdin24/Downloads/KakaoTalk_Video_2025-05-01-15-02-18.mp4: 640x640 (no detections), 136.2ms
video 1/1 (frame 2/1318) /Users/laxdin24/Downloads/KakaoTalk_Video_2025-05-01-15-02-18.mp4: 640x640 (no detections), 69.4ms
video 1/1 (frame 3/1318) /Users/laxdin24/Downloads/KakaoTalk_Video_2025-05-01-15-02-18.mp4: 640x640 (no detections), 45.7ms
video 1/1 (frame 4/1318) /Users/laxdin24/Downloads/KakaoTalk_Video_2025-05-01-15-02

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: ultralytics.engine.results.Keypoints object
 masks: None
 names: {0: 'climber'}
 obb: None
 orig_img: array([[[153, 155, 150],
         [153, 155, 150],
         [153, 155, 150],
         ...,
         [ 93,  98, 100],
         [ 93,  98, 100],
         [ 93,  98, 100]],
 
        [[153, 155, 150],
         [153, 155, 150],
         [153, 155, 150],
         ...,
         [ 93,  98, 100],
         [ 93,  98, 100],
         [ 93,  98, 100]],
 
        [[153, 155, 150],
         [153, 155, 150],
         [153, 155, 150],
         ...,
         [ 93,  98, 100],
         [ 93,  98, 100],
         [ 93,  98, 100]],
 
        ...,
 
        [[176, 169, 171],
         [176, 169, 171],
         [176, 169, 171],
         ...,
         [143, 136, 137],
         [143, 136, 137],
         [143, 136, 137]],
 
        [[175, 168, 169],
         [175, 168, 169],
         [175, 168